## Zope Object Database


https://zodb.org/en/latest/

### Mise en service

<p id="install" style="font-size:120%">
Installation
</p>

<pre style="background-color:#f0f0f0; border: 1px solid #ccc; margin:0; padding:0.5em; font-size:1em; margin-top:1.33em">
pip install ZODB
</pre>

<p id="usage" style="font-size:120%">
Usage
</p>

In [1]:
import ZODB

<div style="font-size:120%">
Choix du mécanisme de stockage : ici un fichier sur disque.
</div>

In [2]:
import ZODB.FileStorage, zc.lockfile

In [3]:
try:
    storage = ZODB.FileStorage.FileStorage('zodb/zodb_demo.fs')
except zc.lockfile.LockError:
    print('lock error')

<div style="font-size:120%">
On observe l'apparition d'un certain nombre de fichiers associés à la base de données :
</div>
<img src="zodb_demo.png" style="margin-left:0">

### Connexion

<p style="font-size:120%">
Instantiation du moteur de base de données :
</p>

In [4]:
db = ZODB.DB(storage)

<div style="font-size:120%">
Ouverture d'une connexion, et récupération de la racine de la base de données
</div>

In [5]:
conn = db.open()
root = conn.root()

<div id="persistent"/>

### Classe persistante

In [6]:
import persistent

In [7]:
class Person(persistent.Persistent):

    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name= last_name
        self.unique = "{}:{} {}".format(
            self.__class__.__name__, self.first_name, self.last_name )
        
    #def __str__(self):
    #   return self.unique

<div id="create"/>

### Enregistrement d'un objet

In [8]:
import transaction

raymond = Person('Raymond','Deubaze')

root.raymond = raymond
transaction.commit()
conn.close()

In [9]:
print(raymond.unique, raymond)

Person:Raymond Deubaze <__main__.Person object at 0x0000027C94E4E6D8 oid 0x23 in <Connection at 27c94ee3198>>


<div id="read"/>

### Récupération d'un objet

In [10]:
db = ZODB.DB(storage)
conn = db.open()
root = conn.root()
person = root.raymond

print(person.unique, person)
conn.close()

Person:Raymond Deubaze <__main__.Person object at 0x0000027C94E4E828 oid 0x23 in <Connection at 27c94ee3dd8>>


<div id="oid" />

### Identifiant unique

In [11]:
id = person._p_oid
print(id)

oid = int.from_bytes(id,byteorder='big')
print(oid)

b'\x00\x00\x00\x00\x00\x00\x00#'
35


<div id="get_by_oid" style="font-size:120%">
On peut récupérer un objet à partir de son identifiant
</div>

In [12]:
print(oid.to_bytes(8, byteorder='big'))

b'\x00\x00\x00\x00\x00\x00\x00#'


In [13]:
conn2 = db.open()
same_person = conn2.get(id)
print(same_person.unique, same_person )

Person:Raymond Deubaze <__main__.Person object at 0x0000027C94E4E828 oid 0x23 in <Connection at 27c94ee3dd8>>


<div style="font-size:120%">
Bizarre : on récupère <b>le même</b> objet, si on se sert d'une nouvelle connexion
alors que la première a été fermée...
</div>

In [14]:
assert same_person == person
assert conn.root().raymond == conn2.root().raymond

<div style="font-size:120%">
Si par contre un crée une seconde connexion alors que la première n'est pas fermée, on récupère un clone...
</div>

In [15]:
conn3 = db.open()
third_person = conn3.get(id)
print(third_person.unique, third_person )

Person:Raymond Deubaze <__main__.Person object at 0x0000027C94E4E908 oid 0x23 in <Connection at 27c94ee34e0>>


In [16]:
try:
    assert third_person == same_person
except AssertionError:
    print("pas la même personne")

assert not conn2.root().raymond == conn3.root().raymond

pas la même personne


In [17]:
person.new_attribute = "test"
print(person.__dict__)
print(same_person.__dict__)
print(third_person.__dict__)

{'first_name': 'Raymond', 'last_name': 'Deubaze', 'unique': 'Person:Raymond Deubaze', 'new_attribute': 'test'}
{'first_name': 'Raymond', 'last_name': 'Deubaze', 'unique': 'Person:Raymond Deubaze', 'new_attribute': 'test'}
{'first_name': 'Raymond', 'last_name': 'Deubaze', 'unique': 'Person:Raymond Deubaze'}


In [18]:
print(conn.transaction_manager)
print(conn2.transaction_manager)
print(conn3.transaction_manager)

In [19]:
transaction.abort()
conn2.close()
conn3.close()

<div style="font-size:120%">
Si on récupère un objet de même id via un autre moteur, on obtient également un clone
</div>

In [20]:
db2 = ZODB.DB(storage)
conn3= db2.open()
clone = conn3.get(person._p_oid)
print(clone)

<__main__.Person object at 0x0000027C94E4E978 oid 0x23 in <Connection at 27c94efda58>>


In [21]:
try:
    assert clone == person
except AssertionError:
    print("le clone n'est pas l'objet")

le clone n'est pas l'objet


In [22]:
conn3.close()

<div id="update" />

### Update

<p style="font-size:120%">
Si l'on modifie les attributs d'un objet, le prochain commit enregistre automatiquement l'objet modifié
</p>

In [23]:
class Office:
    def __init__(self, building, room):
        self.building = building
        self.room = room

conn = db.open()

root = conn.root()
raymond = root.raymond
raymond.office = Office('H9','Aquitaine')

transaction.commit()

In [24]:
conn2 = db2.open()
clone = conn2.get(raymond._p_oid)
assert not clone == raymond
print(clone.office)

<div style="font-size:120%">
Le passage d'information au clone n'est pas automatique...
</div>

In [25]:
from random import random

conn = db.open()

root = conn.root()
raymond = root.raymond
raymond.random = random()
transaction.commit()

print(raymond.random)

# le clone n'est pas mis à jour
try:
    print(clone.random)
except AttributeError:
    print('None')

# même en essayant de le relire
clone = conn2.get(raymond._p_oid)
try:
    print(clone.random)
except AttributeError:
    print('None')

clone = conn2.root().raymond
try:
    print(clone.random)
except AttributeError:
    print('None')
conn2.close()

# ou en créant une nouvelle connexion
conn3 = db2.open()
clone = conn3.root().raymond
try:
    print(clone.random)
except AttributeError:
    print('None')
conn3.close()

# Il faut un nouveau moteur pour relire la base de donnée
db2 = ZODB.DB(storage)
conn2 = db2.open()
clone = conn2.root().raymond
print(clone.random)
conn2.close()

0.9800159224768171
None
None
None
None
0.9800159224768171


Fonction pour vérifier si un objet a bien été modifié dans la base

In [26]:
import copy

def get_from_db(oid):
    with ZODB.DB(storage).transaction() as conn:
        obj = conn.get(oid)
    
        # deepcopy pour éviter une erreur du genre
        # Shouldn't load state for ... when the connection is closed
        return copy.deepcopy(obj)

<div id="p_changed"/>

### Modification d'objets composés



<p style="font-size:120%">
Attention aux listes et aux dictionnaires, car l'ajout d'un élément ne modifie pas la liste elle-même !
</p>

In [27]:
raymond.friends = [ Person('Jean','Bombeur'), Person('Alex','Terrieur') ]
print([f.unique for f in raymond.friends])

transaction.commit()

['Person:Jean Bombeur', 'Person:Alex Terrieur']


In [28]:
clone = get_from_db(raymond._p_oid)
print([f.unique for f in clone.friends])

['Person:Jean Bombeur', 'Person:Alex Terrieur']


In [29]:
# On ajoute un élément à la liste
raymond.friends.append(Person('Anna','Conda'))
transaction.commit()

# La modification n'a pas été vue par ZODB,
# car les attributs de Raymond n'ont pas été modifiés
clone = get_from_db(raymond._p_oid)
print([f.unique for f in clone.friends])

['Person:Jean Bombeur', 'Person:Alex Terrieur']


<div style="font-size:120%">
Il faut signifier à ZODB que l'objet a été modifié :
</div>

In [30]:
raymond._p_changed = True
transaction.commit()

clone = get_from_db(raymond._p_oid)
print([f.unique for f in clone.friends])

['Person:Jean Bombeur', 'Person:Alex Terrieur', 'Person:Anna Conda']


<div id="setter_changed" style="font-size:120%">
Exemple d'implémentation au sein de la classe Person
</div>

In [31]:
class Person(persistent.Persistent):

    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name= last_name
        self.unique = "{}:{} {}".format(
            self.__class__.__name__, self.first_name, self.last_name )
        self.friends = []
        
    def add_friend(self,f):
        self.friends.append(f)
        self._p_changed = True

In [32]:
conn = db.open()
root = conn.root()
raymond = root.raymond

raymond.add_friend(Person('Jean','Aymard'))
transaction.commit()

In [33]:
clone = get_from_db(raymond._p_oid)
print([f.unique for f in clone.friends])

['Person:Jean Bombeur', 'Person:Alex Terrieur', 'Person:Anna Conda', 'Person:Jean Aymard']
